In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [ ]:
import xarray as xr
import pandas as pd

# List of all your .nc4 file paths
file_paths = [
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240101-S000000-E235959.01.V07B.HDF5.nc4",
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240201-S000000-E235959.02.V07B.HDF5.nc4",
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240301-S000000-E235959.03.V07B.HDF5.nc4",
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240601-S000000-E235959.06.V07B.HDF5.nc4",
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240701-S000000-E235959.07.V07B.HDF5.nc4",  # Fixed filename
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240801-S000000-E235959.08.V07B.HDF5.nc4",
    "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/3B-MO.MS.MRG.3IMERG.20240901-S000000-E235959.09.V07B.HDF5.nc4"
]

# Open multiple files and merge along the time dimension
ds_list = [xr.open_dataset(f, decode_times=True) for f in file_paths]  # Using decode_times=True
merged_ds = xr.concat(ds_list, dim="time")  # Merging along time axis

# Close individual datasets to free memory
for ds in ds_list:
    ds.close()

# Print merged dataset info
print(merged_ds)


<xarray.Dataset> Size: 726MB
Dimensions:                         (time: 7, lon: 3600, lat: 1800)
Coordinates:
  * lat                             (lat) float32 7kB -89.95 -89.85 ... 89.95
  * lon                             (lon) float32 14kB -179.9 -179.9 ... 179.9
  * time                            (time) object 56B 2024-01-01 00:00:00 ......
Data variables:
    precipitationQualityIndex       (time, lon, lat) float32 181MB nan ... nan
    precipitation                   (time, lon, lat) float32 181MB nan ... nan
    probabilityLiquidPrecipitation  (time, lon, lat) float32 181MB 0.0 ... 3.0
    randomError                     (time, lon, lat) float32 181MB nan ... nan
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-MONTH/07;\nDOIa...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimit

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

# Convert cftime.DatetimeJulian to pandas datetime
time_values = merged_ds['time'].values  # Extract time values

# Convert to string, then parse to datetime
converted_time = pd.to_datetime(time_values.astype(str))

# Assign back to dataset
merged_ds = merged_ds.assign_coords(time=converted_time)

# Print first few time values to verify
print(merged_ds['time'].values)


['2024-01-01T00:00:00.000000000' '2024-02-01T00:00:00.000000000'
 '2024-03-01T00:00:00.000000000' '2024-06-01T00:00:00.000000000'
 '2024-07-01T00:00:00.000000000' '2024-08-01T00:00:00.000000000'
 '2024-09-01T00:00:00.000000000']


In [ ]:
merged_ds.to_netcdf("merged_dataset.nc4")  # Save merged file


In [ ]:
# Save the merged dataset with corrected time to a new .nc4 file
merged_ds.to_netcdf("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/merged_dataset.nc4")

print("Processed dataset saved successfully!")


Processed dataset saved successfully!


In [ ]:
print(merged_ds.variables)


Frozen({'lon': <xarray.IndexVariable 'lon' (lon: 3600)> Size: 14kB
array([-179.95   , -179.85   , -179.75   , ...,  179.75   ,  179.84999,
        179.95   ], dtype=float32)
Attributes:
    DimensionNames:  lon
    Units:           degrees_east
    units:           degrees_east
    standard_name:   longitude
    LongName:        Longitude at the center of\n\t\t\t0.10 degree grid inter...
    bounds:          lon_bnds
    axis:            X
    origname:        lon
    fullnamepath:    /Grid/lon, 'lat': <xarray.IndexVariable 'lat' (lat: 1800)> Size: 7kB
array([-89.95, -89.85, -89.75, ...,  89.75,  89.85,  89.95], dtype=float32)
Attributes:
    DimensionNames:  lat
    Units:           degrees_north
    units:           degrees_north
    standard_name:   latitude
    LongName:        Latitude at the center of\n\t\t\t0.10 degree grid interv...
    bounds:          lat_bnds
    axis:            Y
    origname:        lat
    fullnamepath:    /Grid/lat, 'precipitationQualityIndex': <xarray.

In [ ]:
ds_india = merged_ds.sel(lat=slice(6, 38), lon=slice(68, 98))


In [ ]:
print(ds_india)


<xarray.Dataset> Size: 11MB
Dimensions:                         (lon: 300, lat: 320, time: 7)
Coordinates:
  * lon                             (lon) float32 1kB 68.05 68.15 ... 97.95
  * lat                             (lat) float32 1kB 6.05 6.15 ... 37.85 37.95
  * time                            (time) datetime64[ns] 56B 2024-01-01 ... ...
Data variables:
    precipitationQualityIndex       (time, lon, lat) float32 3MB ...
    precipitation                   (time, lon, lat) float32 3MB ...
    probabilityLiquidPrecipitation  (time, lon, lat) float32 3MB ...
    randomError                     (time, lon, lat) float32 3MB ...
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-MONTH/07;\nDOIa...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:            

In [ ]:
print(ds_india["precipitation"].min().values, ds_india["precipitation"].max().values)


0.0 3.625000238418579


In [ ]:
import pandas as pd

# Convert xarray dataset to pandas dataframe
df = ds_india.to_dataframe().reset_index()

# Display the first few rows
print(df.head())


         lon   lat       time  precipitationQualityIndex  precipitation  \
0  68.049995  6.05 2024-01-01                       3.94          0.190   
1  68.049995  6.05 2024-02-01                       3.27          0.043   
2  68.049995  6.05 2024-03-01                       2.74          0.009   
3  68.049995  6.05 2024-06-01                       3.76          0.149   
4  68.049995  6.05 2024-07-01                       4.01          0.302   

   probabilityLiquidPrecipitation  randomError  
0                           100.0        0.045  
1                           100.0        0.020  
2                           100.0        0.010  
3                           100.0        0.039  
4                           100.0        0.062  


In [ ]:
df_grouped = df.groupby("time")["precipitation"].mean().reset_index()

print(df_grouped.head())  # Check the format


        time  precipitation
0 2024-01-01       0.035103
1 2024-02-01       0.017206
2 2024-03-01       0.026772
3 2024-06-01       0.328494
4 2024-07-01       0.427952


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_grouped["precipitation"] = scaler.fit_transform(df_grouped[["precipitation"]])

print(df_grouped.head())  # Normalized data


        time  precipitation
0 2024-01-01       0.043571
1 2024-02-01       0.000000
2 2024-03-01       0.023290
3 2024-06-01       0.757861
4 2024-07-01       1.000000


In [ ]:
# Print all unique time values in dataset
print(df["time"].unique())


<DatetimeArray>
['2024-01-01 00:00:00', '2024-02-01 00:00:00', '2024-03-01 00:00:00',
 '2024-06-01 00:00:00', '2024-07-01 00:00:00', '2024-08-01 00:00:00',
 '2024-09-01 00:00:00']
Length: 7, dtype: datetime64[ns]


In [ ]:
# Group data by month and count available entries
df["month"] = pd.to_datetime(df["time"]).dt.month
month_counts = df.groupby("month")["precipitation"].count()

print(month_counts)  # Print number of entries per month


month
1    96000
2    96000
3    96000
6    96000
7    96000
8    96000
9    96000
Name: precipitation, dtype: int64


/usr/local/lib/python3.11/dist-packages/pandas/core/arrays/datetimes.py:666: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  converted = ints_to_pydatetime(


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Normalize the precipitation values
df_filled["precipitation"] = scaler.fit_transform(df_filled[["precipitation"]])

print(df_filled.head())  # Check normalized values


         lon   lat       time  precipitationQualityIndex  precipitation  \
0  68.049995  6.05 2024-01-01                       3.94       0.052414   
1  68.049995  6.05 2024-02-01                       3.27       0.011862   
2  68.049995  6.05 2024-03-01                       2.74       0.002483   
3  68.049995  6.05 2024-06-01                       3.76       0.041103   
4  68.049995  6.05 2024-07-01                       4.01       0.083310   

   probabilityLiquidPrecipitation  randomError  
0                           100.0        0.045  
1                           100.0        0.020  
2                           100.0        0.010  
3                           100.0        0.039  
4                           100.0        0.062  


In [ ]:
import numpy as np

def create_sequences(data, seq_length=3):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Past 'seq_length' values
        y.append(data[i+seq_length])    # Next time step
    return np.array(X), np.array(y)

# Prepare sequences for LSTM
seq_length = 3  # Lookback window (adjustable)
X, y = create_sequences(df_filled["precipitation"].values, seq_length)

print(f"X shape: {X.shape}, y shape: {y.shape}")


X shape: (671997, 3), y shape: (671997,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"Training size: {len(X_train)}, Testing size: {len(X_test)}")


Training size: 537597, Testing size: 134400


In [ ]:
# Reshape data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")


X_train shape: (537597, 3, 1), X_test shape: (134400, 3, 1)


In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(3, 1)),  # Ensure (timesteps, features)
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 3, 50)               │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 3, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (537597, 3, 1)
y_train shape: (537597, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(3, 1)),  # Ensure (timesteps, features)
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Model summary
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='mse') # Example using 'adam' optimizer and 'mse' loss
                                           # You can adjust these based on your needs

history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 3, 50)               │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 3, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 80s 9ms/step - loss: 0.0088 - val_loss: 0.0070
Epoch 2/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 69s 8ms/step - loss: 0.0037 - val_loss: 0.0071
Epoch 3/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 75s 9ms/step - loss: 0.0036 - val_loss: 0.0062
Epoch 4/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - loss: 0.0033 - val_loss: 0.0063
Epoch 5/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 74s 9ms/step - loss: 0.0031 - val_loss: 0.0059
Epoch 6/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 73s 9ms/step - loss: 0.0030 - val_loss: 0.0062
Epoch 7/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 83s 9ms/step - loss: 0.0030 - val_loss: 0.0065
Epoch 8/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 77s 8ms/step - loss: 0.0030 - val_loss: 0.0064
Epoch 9/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 83s 8ms/step - loss: 0.0029 - val_loss: 0.0064
Epoch 10/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - loss: 0.0029 - val_loss: 0.0056
Epoch 11/20
8400/8400 ━━━━━━━━━━━━━━━━━━━━ 68s 8ms/step - loss: 0.0029 - val_loss: 0.0062
Epoch 12/20
8400/84

In [ ]:
model.save("flood_prediction_model.keras")
print("✅ Model saved successfully as flood_prediction_model.h5")


✅ Model saved successfully as flood_prediction_model.h5


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("flood_prediction_model.keras")
print("✅ Model loaded successfully!")


✅ Model loaded successfully!


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
evaluation_metrics = model.evaluate(X_test, y_test)
print(f"📊 Model Evaluation: {evaluation_metrics}")


4200/4200 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0066
📊 Model Evaluation: 0.006359281949698925


In [ ]:
evaluation_metrics = model.evaluate(X_test, y_test)

if isinstance(evaluation_metrics, (list, tuple)):
    loss, mae = evaluation_metrics
    print(f"📊 Model Evaluation: Loss = {loss:.4f}, MAE = {mae:.4f}")
else:
    loss = evaluation_metrics  # If only loss is returned
    print(f"📊 Model Evaluation: Loss = {loss:.4f}")


4200/4200 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0066
📊 Model Evaluation: Loss = 0.0064


In [ ]:
y_pred = model.predict(X_test)
print(y_pred[:10])  # Show first 10 predictions


4200/4200 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
[[0.02083854]
 [0.110149  ]
 [0.01073393]
 [0.04413974]
 [0.07634331]
 [0.05263359]
 [0.06822101]
 [0.09858719]
 [0.01267175]
 [0.00389596]]


In [ ]:
def categorize_flood_severity(predictions):
    categories = []
    for value in predictions:
        if value < 0.3:
            categories.append("Low")
        elif 0.3 <= value < 0.7:
            categories.append("Medium")
        else:
            categories.append("Extreme")
    return categories

flood_severity = categorize_flood_severity(y_pred.flatten())
print(flood_severity[:10])  # Show first 10 classified results


['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low']


In [ ]:
# Step 1: Install & Import Required Libraries
import xarray as xr
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 2: Load the .nc4 dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/merged_dataset.nc4"  # Replace with your file path
ds = xr.open_dataset(file_path)

# Step 3: Extract India-specific data (Lat: 6N-38N, Lon: 68E-98E)
lat_min, lat_max = 6.0, 38.0
lon_min, lon_max = 68.0, 98.0

ds_india = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

# Step 4: Convert Time Format (Unix to Datetime)
ds_india["time"] = pd.to_datetime(ds_india["time"].values)

# Step 5: Handle Missing Values (-9999.9, -9999)
precip = ds_india["precipitation"].values
missing_values = [-9999.9, -9999]

# Replace missing values with NaN
for mv in missing_values:
    precip[precip == mv] = np.nan

# Fill NaN using median interpolation
median_value = np.nanmedian(precip)
precip = np.nan_to_num(precip, nan=median_value)

# Step 6: Detect & Handle Outliers using IQR
Q1 = np.percentile(precip, 25)
Q3 = np.percentile(precip, 75)
IQR = Q3 - Q1

# Define outlier threshold
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Replace outliers with median value
precip = np.where((precip < lower_bound) | (precip > upper_bound), median_value, precip)

# Step 7: Normalize Precipitation Data (Min-Max Scaling)
scaler = MinMaxScaler()
precip_reshaped = precip.reshape(-1, 1)  # Reshape for scaling
precip_scaled = scaler.fit_transform(precip_reshaped).reshape(precip.shape)

# Step 8: Store processed data back in xarray dataset
ds_india["precipitation"].values = precip_scaled

# Step 9: Save Preprocessed Dataset in .nc4 Format
output_path = "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/processed_india_floods.nc4"
ds_india.to_netcdf(output_path)

print(f"✅ Preprocessing completed! Saved as {output_path}")


✅ Preprocessing completed! Saved as /content/drive/MyDrive/Colab Notebooks/Nasa Floods/processed_india_floods.nc4


In [ ]:
print(ds_india)

<xarray.Dataset> Size: 11MB
Dimensions:                         (lon: 300, lat: 320, time: 7)
Coordinates:
  * lon                             (lon) float32 1kB 68.05 68.15 ... 97.95
  * lat                             (lat) float32 1kB 6.05 6.15 ... 37.85 37.95
  * time                            (time) datetime64[ns] 56B 2024-01-01 ... ...
Data variables:
    precipitationQualityIndex       (time, lon, lat) float32 3MB ...
    precipitation                   (time, lon, lat) float32 3MB 0.253 ... 0....
    probabilityLiquidPrecipitation  (time, lon, lat) float32 3MB ...
    randomError                     (time, lon, lat) float32 3MB ...
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-MONTH/07;\nDOIa...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time
    history:

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
from sklearn.preprocessing import MinMaxScaler

# Load dataset
ds = xr.open_dataset('/content/drive/MyDrive/Colab Notebooks/Nasa Floods/processed_india_floods.nc4')
df = ds.to_dataframe().reset_index()

# ✅ 1️⃣ Temporal Features
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year

# Map seasons
season_map = {6: 'monsoon', 7: 'monsoon', 8: 'monsoon', 9: 'monsoon',
              10: 'post-monsoon', 11: 'post-monsoon',
              12: 'winter', 1: 'winter', 2: 'winter',
              3: 'pre-monsoon', 4: 'pre-monsoon', 5: 'pre-monsoon'}
df['season'] = df['month'].map(season_map)

# ✅ 2️⃣ Region Classification (Improved)
def classify_region(lat, lon):
    if lat > 25: return 'North'
    elif lat > 20 and lon > 80: return 'East'
    elif lat > 20: return 'West'
    else: return 'South'

df['region'] = df.apply(lambda row: classify_region(row['lat'], row['lon']), axis=1)

# ✅ 3️⃣ Rainfall-Based Features
df['precip_rolling_3m'] = df.groupby(['lat', 'lon'])['precipitation'].transform(lambda x: x.rolling(3, min_periods=1).mean())
df['precip_anomaly'] = df.groupby(['lat', 'lon'])['precipitation'].transform(lambda x: x - x.mean())

# ✅ 4️⃣ Heavy Rainfall Flag (Above 90th Percentile for more sensitivity)
df['heavy_rain'] = (df['precipitation'] > df['precipitation'].quantile(0.90)).astype(int)

# ✅ 5️⃣ Flood Risk Classification (Revised Thresholds)
def classify_flood_risk(precip):
    if precip < 0.2: return 0  # Low risk
    elif precip < 0.6: return 1  # Medium risk
    else: return 2  # High risk

df['flood_risk'] = df['precipitation'].apply(classify_flood_risk)

# ✅ 6️⃣ Normalize AFTER Feature Engineering
scaler = MinMaxScaler()
df[['precipitation', 'precip_rolling_3m', 'precip_anomaly']] = scaler.fit_transform(df[['precipitation', 'precip_rolling_3m', 'precip_anomaly']])

# ✅ 7️⃣ Save Processed Data
ds_new = xr.Dataset.from_dataframe(df)
ds_new.to_netcdf("processed_india_floods.nc4")

print("✅ Preprocessing Completed! Data saved as processed_india_floods.nc4")


✅ Data Prepared for BiLSTM Training!
X_train shape: (307200, 3, 1), y_train shape: (307200, 3)


In [ ]:
import xarray as xr
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load Processed Dataset
data_path = "processed_india_floods.nc4"
dataset = xr.open_dataset(data_path)

#  Convert Xarray to Pandas
df = dataset.to_dataframe().reset_index()

#  Define Sequence Length for BiLSTM
seq_length = 3  # Using past 3 time steps for prediction

X, y = [], []

#  Convert Data to Sequences
for lat_lon, group in df.groupby(['lat', 'lon']):
    precip_series = group['precipitation'].values
    flood_series = group['flood_risk'].values

    for i in range(len(precip_series) - seq_length):
        X.append(precip_series[i:i+seq_length])  # Input: past 3 time steps
        y.append(flood_series[i+seq_length])  # Target: next time step

#  Convert to NumPy Arrays
X, y = np.array(X), np.array(y)
X = X.reshape(X.shape[0], X.shape[1], 1)  # Shape: (samples, time_steps, features)

#  Convert Labels to Categorical (One-Hot Encoding)
y = to_categorical(y, num_classes=3)

#  Split Data into Train & Test
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

print(" Data Prepared for BiLSTM Training!")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


✅ Data Prepared for BiLSTM Training!
X_train shape: (307200, 3, 1), y_train shape: (307200, 3)


In [ ]:
print(df['flood_risk'].value_counts())


flood_risk
0    459599
1    147752
2     64649
Name: count, dtype: int64


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

#  Define BiLSTM Model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(seq_length, 1))),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dense(16, activation="relu"),
    Dropout(0.2),
    Dense(3, activation="softmax")  # 3 Flood Risk Levels: Low, Medium, High
])

# Compile Model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train Model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

print(" BiLSTM Model Training Complete!")


Epoch 1/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 116s 11ms/step - accuracy: 0.4873 - loss: 1.0123 - val_accuracy: 0.8005 - val_loss: 0.7776
Epoch 2/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 103s 11ms/step - accuracy: 0.5290 - loss: 0.9555 - val_accuracy: 0.8404 - val_loss: 0.6466
Epoch 3/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 144s 11ms/step - accuracy: 0.5512 - loss: 0.9188 - val_accuracy: 0.8056 - val_loss: 0.6697
Epoch 4/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 139s 11ms/step - accuracy: 0.5601 - loss: 0.9009 - val_accuracy: 0.7435 - val_loss: 0.6200
Epoch 5/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 143s 11ms/step - accuracy: 0.5663 - loss: 0.8909 - val_accuracy: 0.7591 - val_loss: 0.6326
Epoch 6/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 103s 11ms/step - accuracy: 0.5713 - loss: 0.8827 - val_accuracy: 0.6900 - val_loss: 0.6368
Epoch 7/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 141s 11ms/step - accuracy: 0.5735 - loss: 0.8784 - val_accuracy: 0.7139 - val_loss: 0.6339
Epoch 8/15
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 143s 11ms/step - accuracy: 

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Nasa Floods/predict_flood.keras')

In [ ]:
import tensorflow as tf
import numpy as np
import xarray as xr

# ✅ Load the Previous Model
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/predict_flood.keras")  # Load your last saved model


# ✅ Load Processed Dataset
data_path = "processed_india_floods.nc4"
dataset = xr.open_dataset(data_path)

# ✅ Convert Xarray to Pandas
df = dataset.to_dataframe().reset_index()

# ✅ Define Sequence Length for BiLSTM
seq_length = 3  # Using past 3 time steps for prediction

X, y = [], []

# ✅ Convert Data to Sequences
for lat_lon, group in df.groupby(['lat', 'lon']):
    precip_series = group['precipitation'].values
    flood_series = group['flood_risk'].values

    for i in range(len(precip_series) - seq_length):
        X.append(precip_series[i:i+seq_length])  # Input: past 3 time steps
        y.append(flood_series[i+seq_length])  # Target: next time step

# ✅ Convert to NumPy Arrays
X, y = np.array(X), np.array(y)
X = X.reshape(X.shape[0], X.shape[1], 1)  # Shape: (samples, time_steps, features)

# ✅ Convert Labels to Categorical (One-Hot Encoding)
y = to_categorical(y, num_classes=3)

# ✅ Split Data into Train & Test
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

print("✅ Data Prepared for BiLSTM Training!")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")



✅ Data Prepared for BiLSTM Training!
X_train shape: (307200, 3, 1), y_train shape: (307200, 3)


In [ ]:
# ✅ Resume Training on the Same Data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# ✅ Save the Improved Model
model.save("improved_flood_bilstm.h5")
print("✅ Training Continued & Model Saved as improved_flood_bilstm.h5")


Epoch 1/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 64s 13ms/step - accuracy: 0.5933 - loss: 0.8457 - val_accuracy: 0.6677 - val_loss: 0.6795
Epoch 2/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.5925 - loss: 0.8462 - val_accuracy: 0.6997 - val_loss: 0.6641
Epoch 3/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 85s 14ms/step - accuracy: 0.5940 - loss: 0.8455 - val_accuracy: 0.6639 - val_loss: 0.6854
Epoch 4/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 61s 13ms/step - accuracy: 0.5939 - loss: 0.8440 - val_accuracy: 0.7054 - val_loss: 0.6575
Epoch 5/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 63s 13ms/step - accuracy: 0.5945 - loss: 0.8420 - val_accuracy: 0.6744 - val_loss: 0.6754
Epoch 6/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 86s 14ms/step - accuracy: 0.5941 - loss: 0.8433 - val_accuracy: 0.6976 - val_loss: 0.6605
Epoch 7/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 83s 14ms/step - accuracy: 0.5960 - loss: 0.8396 - val_accuracy: 0.6676 - val_loss: 0.7054
Epoch 8/10
4800/4800 ━━━━━━━━━━━━━━━━━━━━ 81s 14ms/step - accuracy: 0.5970 -

✅ Training Continued & Model Saved as improved_flood_bilstm.h5


In [ ]:
# ✅ Load the New Model
new_model = tf.keras.models.load_model("improved_flood_bilstm.h5")

# ✅ Evaluate on Test Data
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f"✅ New Test Accuracy: {accuracy:.4f}")


2400/2400 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.6837 - loss: 0.7209
✅ New Test Accuracy: 0.7152


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
from tensorflow.keras.models import load_model

# Load the processed dataset
data_path = "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/processed_india_floods.nc4"  # Update with correct path
ds = xr.open_dataset(data_path)
df = ds.to_dataframe().reset_index()

# Load the trained BiLSTM model
model_path = "/content/drive/MyDrive/Colab Notebooks/Nasa Floods/improved_flood_bilstm.h5"  # Update with actual model path
model = load_model(model_path)

# Sequence length used in training
seq_length = 3

# Function to create sequences for prediction
def create_sequences(df, seq_length):
    X_pred = []
    coords = []  # To store lat, lon, and time

    grouped = df.groupby(['lat', 'lon'])
    for (lat, lon), group in grouped:
        group = group.sort_values('time')

        precip_series = group['precipitation'].values

        for i in range(len(precip_series) - seq_length):
            X_pred.append(precip_series[i:i+seq_length])
            coords.append((lat, lon, group['time'].iloc[i + seq_length]))

    return np.array(X_pred), coords

# Create sequences
X_pred, coords = create_sequences(df, seq_length)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[1], 1)

# Generate predictions
predictions = model.predict(X_pred)
predicted_risk = np.argmax(predictions, axis=1)  # Convert to categorical values (0: Low, 1: Medium, 2: High)

# Convert to DataFrame
prediction_df = pd.DataFrame(coords, columns=['lat', 'lon', 'time'])
prediction_df['predicted_risk'] = predicted_risk

# Merge with original data for additional insights
final_df = prediction_df.merge(df, on=['lat', 'lon', 'time'], how='left')

# Save to CSV
csv_path = "flood_predictions.csv"
final_df.to_csv(csv_path, index=False)

print(f"✅ Predictions saved to {csv_path}")


12000/12000 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step
✅ Predictions saved to flood_predictions.csv


In [ ]:
print("Shape before reshaping:", dataset['precipitation'][-seq_length:].values.shape)


Shape before reshaping: (3, 320, 300)


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load India states shapefile
india_states = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/india_state.geojson")
india_states = india_states.to_crs(epsg=4326)  # Ensure correct CRS

# Load flood prediction CSV
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/flood_predictions.csv")

# Convert lat/lon to geometric points
df["geometry"] = df.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Spatial join: assign state names to each flood point
gdf = gpd.sjoin(gdf, india_states, how="left", predicate="within")
gdf.rename(columns={"NAME_1": "State"}, inplace=True)

# Count how many points got a state assigned
print(f"Total flood points: {len(gdf)}")
print(f"Flood points inside India: {gdf['State'].notna().sum()}")
print("Sample points inside India:")
print(gdf[gdf["State"].notna()][["lat", "lon", "State"]].head())


Total flood points: 384000
Flood points inside India: 111472
Sample points inside India:
        lat    lon                State
10632  6.85  93.85  Andaman and Nicobar
10633  6.85  93.85  Andaman and Nicobar
10634  6.85  93.85  Andaman and Nicobar
10635  6.85  93.85  Andaman and Nicobar
11828  6.95  93.75  Andaman and Nicobar


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree

# **1️⃣ Load India states shapefile**
india_states = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/india_state.geojson")

# Convert to projected CRS for accurate centroid calculation
india_states = india_states.to_crs(epsg=3857)

# Compute centroids in projected CRS
state_centroids = india_states.geometry.centroid

# Convert back to WGS 84 (EPSG:4326) for mapping
india_states = india_states.to_crs(epsg=4326)
state_centroids = state_centroids.to_crs(epsg=4326)

# **2️⃣ Load Flood Data (CSV)**
flood_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/flood_predictions.csv")

# **3️⃣ Convert lat/lon to geometry (GeoDataFrame)**
flood_gdf = gpd.GeoDataFrame(flood_df,
                             geometry=gpd.points_from_xy(flood_df.lon, flood_df.lat),
                             crs="EPSG:4326")

# **🛠 FIX: Add "State" column if missing**
if "State" not in flood_gdf.columns:
    flood_gdf["State"] = None  # Create an empty State column

# **4️⃣ Build KDTree for Nearest State Lookup**
tree = cKDTree(list(zip(state_centroids.x, state_centroids.y)))

def find_nearest_state_fast(point):
    """Find the nearest state for ocean points."""
    _, idx = tree.query([point.x, point.y])
    return india_states.iloc[idx]["NAME_1"]  # Get nearest state name

# **5️⃣ Find Missing States**
missing_state = flood_gdf[flood_gdf["State"].isna()]

# Apply nearest state function only to missing rows
missing_state["State"] = missing_state["geometry"].apply(find_nearest_state_fast)

# Merge back into main dataset
flood_gdf.update(missing_state)

# **6️⃣ Save the Updated Data**
flood_gdf.to_csv("flood_predictions_with_states.csv", index=False)
print("✅ Nearest states assigned & saved.")


✅ Nearest states assigned & saved.


In [ ]:
print(flood_gdf.head())  # Check first few rows
print(flood_gdf["State"].unique())  # List all assigned states


    lat        lon        time  predicted_risk  precipitationQualityIndex  \
0  6.05  68.049995  2024-06-01               0                       3.76   
1  6.05  68.049995  2024-07-01               0                       4.01   
2  6.05  68.049995  2024-08-01               1                       3.97   
3  6.05  68.049995  2024-09-01               1                       3.60   
4  6.05  68.150000  2024-06-01               0                       3.76   

   precipitation  probabilityLiquidPrecipitation  randomError  \
0       0.198402                           100.0        0.039   
1       0.402130                           100.0        0.062   
2       0.318242                           100.0        0.053   
3       0.105193                           100.0        0.027   
4       0.191744                           100.0        0.038   

   precip_3day_avg  precip_7day_avg  flood_risk  precip_zscore  is_anomaly  \
0         0.089214         0.130160           0       0.036226      

In [ ]:
high_risk_floods = flood_gdf[flood_gdf["predicted_risk"] == 2]
print(high_risk_floods[["lat", "lon", "State", "precipitation", "flood_risk"]].head(20))


       lat        lon                State  precipitation  flood_risk
251   6.05  74.250000          Lakshadweep       0.153129           0
255   6.05  74.350000          Lakshadweep       0.183755           0
337   6.05  76.450000               Kerala       0.151798           0
341   6.05  76.549995               Kerala       0.145140           0
345   6.05  76.650000               Kerala       0.145140           0
353   6.05  76.850000               Kerala       0.150466           0
369   6.05  77.250000               Kerala       0.191744           0
409   6.05  78.250000               Kerala       0.201065           0
663   6.05  84.549995           Tamil Nadu       0.082557           0
667   6.05  84.650000           Tamil Nadu       0.081225           0
826   6.05  88.650000  Andaman and Nicobar       0.213049           0
830   6.05  88.750000  Andaman and Nicobar       0.230360           0
910   6.05  90.750000  Andaman and Nicobar       0.404794           0
935   6.05  91.35000

In [ ]:
high_risk_floods.to_csv("high_risk_flood_states.csv", index=False)
print("✅ High-risk flood states saved!")


✅ High-risk flood states saved!


In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from folium.plugins import MarkerCluster

# Load flood predictions CSV
flood_df = pd.read_csv("high_risk_flood_states.csv")

# Load India states shapefile (GeoJSON)
india_states = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Nasa Floods/india_state.geojson")

# Create a Folium map centered in India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Add state boundaries
folium.GeoJson(
    india_states,
    name="State Boundaries",
    style_function=lambda x: {
        "fillColor": "none",
        "color": "blue",
        "weight": 1.5,
    },
    tooltip=folium.GeoJsonTooltip(fields=["NAME_1"], aliases=["State:"]),
).add_to(m)

# Add high-risk flood locations (Red markers)
marker_cluster = MarkerCluster().add_to(m)
for _, row in flood_df.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=folium.Popup(f"State: {row['State']}<br>Precipitation: {row['precipitation']}", parse_html=True),
    ).add_to(marker_cluster)

# Save the map
m.save("high_risk_flood_map.html")
print("✅ High-risk flood map saved as 'high_risk_flood_map.html'")

# Display in Jupyter Notebook (Optional)
m


✅ High-risk flood map saved as 'high_risk_flood_map.html'
Buffered data was truncated after reaching the output size limit.